# TD 8–10 : Moteur de recherche avec Jupyter

Ce notebook regroupe les TD 8, 9 et 10.

- **TD 8** : chargement du corpus, moteur de recherche et interface graphique avec ipywidgets.
- **TD 9–10** : ajout de filtres, analyses du vocabulaire et exploration temporelle des mots.

L’objectif est de proposer une interface interactive pour explorer et analyser le corpus.


## Installation des dépendances nécessaires

In [1]:
# Installation des packages nécessaires
!pip install pandas numpy scipy ipywidgets tqdm matplotlib seaborn plotly --quiet

## Imports des bibliothèques

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
import sys
import matplotlib.pyplot as plt
from collections import defaultdict
# Imports de vos classes personnalisées
from Corpus import Corpus
from Document import Document
from DocumentFactory import DocumentFactory
from SearchEngine import SearchEngine

---
# Partie 1 : Démarrage
## 1.1 - Récupération et chargement du jeu de données

In [3]:
# Chargement du fichier TSV
df_discours = pd.read_csv('corpus.tsv', sep='\t')

print("Aperçu des données :")
print(df_discours.head())
print(f"\nNombre total de documents : {len(df_discours)}")
print(f"\nColonnes disponibles : {list(df_discours.columns)}")

Aperçu des données :
   id                                              titre              auteur  \
0   0       I switched to Machine Learning and I am LOST         Hertz314159   
1   1  Best resources to learn Machine Learning deepl...            vansh596   
2   2           Is a machine learning career still good?            BBBixncx   
3   3                      Getting into Machine Learning   Working_Dress9277   
4   4                 How do you learn machine learning?  ShortImplement4486   

                  date                                                url  \
0  2025-10-29 01:46:28  https://reddit.com/r/learnmachinelearning/comm...   
1  2025-08-18 13:53:27  https://reddit.com/r/learnmachinelearning/comm...   
2  2025-10-12 07:37:00  https://reddit.com/r/AskProgramming/comments/1...   
3  2025-12-07 18:34:38  https://reddit.com/r/learnmachinelearning/comm...   
4  2025-11-18 08:01:41  https://reddit.com/r/computerscience/comments/...   

                                   

## 1.2 - Vérification de la distribution des auteurs

In [4]:
# Distribution des auteurs
print("Distribution des auteurs :")
print(df_discours['auteur'].value_counts())

print(f"\nNombre total d'auteurs uniques : {df_discours['auteur'].nunique()}")

Distribution des auteurs :
Morteza Haghir Chehreghani     2
Sorin61                        2
Sung Whan Yoon                 1
Gustavo Correa Publio          1
space-_-man                    1
Matias Valdenegro-Toro         1
maxwellhill                    1
Willing-Arugula3238            1
Geoffroy Dubourg-Felonneau     1
Thomas M. Moerland             1
egocentric-video               1
Carmen Martin-Turrero          1
gaeioran                       1
Olga Cherednichenko            1
Junaed Younus Khan             1
BBBixncx                       1
ShortImplement4486             1
Cedric De Boom                 1
dstori                         1
TrapWolf                       1
Lun Ai                         1
Diego Granziol                 1
legehjernen                    1
Matthew B. A. McDermott        1
Substantial-Art-2238           1
Affectionate-Army458           1
Felix Mohr                     1
esberat                        1
Cynthia Rudin                  1
Canon_Cowboy    

## 1.3 - Création du corpus avec découpage en phrases

In [5]:
def decouper_en_phrases(texte):
    """Découpe un texte en phrases."""
    phrases = re.split(r'[.!?]+', texte)
    phrases = [p.strip() for p in phrases if p.strip() and len(p.strip()) > 20]
    return phrases

# Création du corpus
corpus_discours = Corpus("Corpus Discours US")

print("Création du corpus avec découpage en phrases...")
phrase_count = 0

for idx, row in tqdm(df_discours.iterrows(), total=len(df_discours), desc="Traitement"):
    texte = str(row['texte'])
    phrases = decouper_en_phrases(texte)
    
    for i, phrase in enumerate(phrases):
        titre_phrase = f"{row['titre']} - Phrase {i+1}"
        
        doc = DocumentFactory.create_document(
            doc_type=row.get('type', 'document').lower(),
            titre=titre_phrase,
            auteur=row['auteur'],
            date=row['date'],
            url=row.get('url', ''),
            texte=phrase
        )
        
        corpus_discours.add(doc)
        phrase_count += 1

print(f"\n{corpus_discours}")
print(f"Nombre total de phrases créées : {phrase_count}")

Traitement: 100%|██████████| 49/49 [00:00<00:00, 2753.24it/s]

Création du corpus avec découpage en phrases...

Corpus 'Corpus Discours US' : 292 documents, 47 auteurs
Nombre total de phrases créées : 292


## 1.4 - Test des fonctions search et concorde

In [6]:
# Test de la fonction search
print("="*80)
print("TEST DE LA FONCTION SEARCH")
print("="*80)

mot_recherche = "learning"
passages = corpus_discours.search(mot_recherche)

print(f"\nPassages contenant '{mot_recherche}' : {len(passages)}")
print(f"\nAffichage des 5 premiers :")
for i, passage in enumerate(passages[:5], 1):
    print(f"\n{i}. ...{passage}...")

TEST DE LA FONCTION SEARCH

Passages contenant 'learning' : 112

Affichage des 5 premiers :

1. ...I switched to Machine Learning and I am LOST Hello everybody, I'm a bit lost and...

2. ... called AI, which focuses on AI logic and Machine Learning I found this really exciting, so even after learn...

3. ... start my journey Best resources to learn Machine Learning deeply in 2–3 months Hey everyone,  I’m planning ...

4. ...pend the next 2–3 months fully focused on Machine Learning I already know Python, NumPy, Pandas, Matplotlib,...

5. ...ly part I really want to dive into now is Machine Learning itself What I’m looking for are resources that go...


In [7]:
# Test de la fonction concorde
print("="*80)
print("TEST DE LA FONCTION CONCORDE")
print("="*80)

expression = "machine"
df_concorde = corpus_discours.concorde(expression, taille_contexte=40)

print(f"\nOccurrences de '{expression}' : {len(df_concorde)}")
print(f"\nAffichage des 10 premières :")
display(df_concorde.head(10))

TEST DE LA FONCTION CONCORDE

Occurrences de 'machine' : 95

Affichage des 10 premières :


,contexte gauche,motif trouvé,contexte droit
0,I switched to,Machine,"Learning and I am LOST Hello everybody,"
1,"alled AI, which focuses on AI logic and",Machine,"Learning I found this really exciting,"
2,tart my journey Best resources to learn,Machine,Learning deeply in 2–3 months Hey every
3,nd the next 2–3 months fully focused on,Machine,"Learning I already know Python, NumPy,"
4,part I really want to dive into now is,Machine,Learning itself What I’m looking for ar
5,", what resources would you rely on Is a",machine,learning career still good Hi I’m 17 an
6,"into the AI industry, specifically as a",machine,learning engineer I have a genuine inte
7,should help me work towards becoming a,machine,learning engineer) If it is not a good
8,do mechanical engineering Getting into,Machine,"Learning Hello, I have a background in"
9,echanical Engineering and want to learn,Machine,Learning from scratch Where should I st


---
# Partie 2 : Utilisation du moteur de recherche
## 2.1 - Import et initialisation du SearchEngine

In [8]:
# Initialisation du moteur de recherche
print("Initialisation du moteur de recherche...\n")

moteur = SearchEngine(corpus_discours)

print("Moteur initialisé avec succès !")
print("\nStatistiques du vocabulaire :")
moteur.afficher_stats_vocab()

Initialisation du moteur de recherche...

Moteur initialisé avec succès !

Statistiques du vocabulaire :
Taille du vocabulaire : 1744 mots

Exemple de mots dans le vocabulaire :
  - a : 136 occurrences, 96 documents
  - abduction : 1 occurrences, 1 documents
  - ability : 5 occurrences, 4 documents
  - able : 1 occurrences, 1 documents
  - about : 3 occurrences, 3 documents
  - academic : 1 occurrences, 1 documents
  - accessibility : 1 occurrences, 1 documents
  - accessible : 1 occurrences, 1 documents
  - according : 2 occurrences, 2 documents
  - accounted : 2 occurrences, 2 documents


## 2.2 - Test avec plusieurs requêtes

In [9]:
# Test avec différentes requêtes
requetes_test = [
    "machine learning",
    "neural network",
    "data science"
]

for requete in requetes_test:
    print("="*80)
    print(f"Requête : '{requete}'")
    print("="*80)
    
    resultats = moteur.search(requete, nb_resultats=5)
    
    if len(resultats) > 0:
        print(f"\n{len(resultats)} résultats trouvés :\n")
        for idx, row in resultats.iterrows():
            print(f"{idx+1}. [{row['type']}] {row['titre']}")
            print(f"   Auteur: {row['auteur']}")
            print(f"   Score: {row['score']:.4f}\n")
    else:
        print("Aucun résultat trouvé.\n")

Requête : 'machine learning'

5 résultats trouvés :

1. [Reddit] This machine learning method... - Phrase 1
   Auteur: NewSomethingUnlocked
   Score: 0.3647

2. [Arxiv] Automatic Machine Learning by Pipeline Synthesis using Model-Based Reinforcement Learning and a Grammar - Phrase 1
   Auteur: Iddo Drori
   Score: 0.2836

3. [Reddit] Machine learning engineer, 31M - Phrase 1
   Auteur: gaeioran
   Score: 0.2719

4. [Reddit] XKCD Machine Learning - Phrase 1
   Auteur: loved_and_held
   Score: 0.2572

5. [Reddit] Machine Learning Magic. - Phrase 1
   Auteur: esberat
   Score: 0.2572

Requête : 'neural network'

5 résultats trouvés :

1. [Arxiv] Automatic Machine Learning by Pipeline Synthesis using Model-Based Reinforcement Learning and a Grammar - Phrase 2
   Auteur: Iddo Drori
   Score: 0.2028

2. [Arxiv] TapNet: Neural Network Augmented with Task-Adaptive Projection for Few-Shot Learning - Phrase 2
   Auteur: Sung Whan Yoon
   Score: 0.1866

3. [Reddit] I switched to Machine Learning 

## 2.3 - Ajout compteur avec tqdm

In [10]:
# Version avec barre de progression
def search_with_progress(self, mots_clefs, nb_resultats=10):
    print("Traitement de la requête...")
    
    vecteur_requete = self._vectoriser_requete(mots_clefs)
    scores = self._similarite_cosinus(vecteur_requete, self.mat_TFxIDF)
    indices_tries = np.argsort(scores)[::-1][:nb_resultats]
    
    resultats = []
    for idx in tqdm(indices_tries, desc="Récupération"):
        if scores[idx] > 0:
            doc = self.corpus.id2doc[idx]
            resultats.append({
                'doc_id': idx,
                'titre': doc.titre,
                'auteur': doc.auteur,
                'date': doc.date,
                'url': doc.url,
                'score': scores[idx],
                'type': doc.getType()
            })
    
    return pd.DataFrame(resultats)

import types
moteur.search_with_progress = types.MethodType(search_with_progress, moteur)

# Test
print("Test avec barre de progression :\n")
resultats = moteur.search_with_progress("machine learning", nb_resultats=10)

if len(resultats) > 0:
    print(f"\n{len(resultats)} résultats :")
    display(resultats[['titre', 'auteur', 'score', 'type']])

Test avec barre de progression :


Récupération: 100%|██████████| 10/10 [00:00<00:00, 60611.33it/s]


Traitement de la requête...

10 résultats :


,titre,auteur,score,type
0,This machine learning method... - Phrase 1,NewSomethingUnlocked,0.364688,Reddit
1,Automatic Machine Learning by Pipeline Synthes...,Iddo Drori,0.283620,Arxiv
2,"Machine learning engineer, 31M - Phrase 1",gaeioran,0.271937,Reddit
3,XKCD Machine Learning - Phrase 1,loved_and_held,0.257243,Reddit
4,Machine Learning Magic. - Phrase 1,esberat,0.257243,Reddit
5,How do you learn machine learning? - Phrase 1,ShortImplement4486,0.229862,Reddit
6,Is machine learning a good career in 2025? - P...,stanley_john,0.223631,Reddit
7,Emotion in Reinforcement Learning Agents and R...,Thomas M. Moerland,0.215450,Arxiv
8,Getting into Machine Learning - Phrase 1,Working_Dress9277,0.212482,Reddit
9,Teaching Uncertainty Quantification in Machine...,Matias Valdenegro-Toro,0.208356,Arxiv


---
# Partie 3 : Interface graphique
## 3.1 - Création des objets graphiques

In [11]:
# Widgets de base
titre_label = widgets.Label(
    value='Moteur de recherche'
)

mots_cles_text = widgets.Text(
    value='',
    placeholder='mot1 mot2 mot3',
    description='Mots clés :',
    layout=widgets.Layout(width='500px')
)

nb_docs_slider = widgets.IntSlider(
    value=10,
    min=1,
    max=50,
    description="Nombre d'articles à extraire :",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

print("Widgets créés !")

Widgets créés !


## 3.2 - Affichage avec VBox

In [12]:
interface_simple = widgets.VBox([
    titre_label,
    mots_cles_text,
    nb_docs_slider
])

display(interface_simple)

## 3.3 - Ajout de l'objet Output

In [13]:
sortie_resultats = widgets.Output()

interface_output = widgets.VBox([
    titre_label,
    mots_cles_text,
    nb_docs_slider,
    sortie_resultats
])

display(interface_output)

## 3.4 - Ajout du bouton

In [14]:
bouton_recherche = widgets.Button(
    description='Rechercher',
    button_style='primary',
    tooltip='Lancer la recherche',
    icon='search'
)

interface_bouton = widgets.VBox([
    titre_label,
    mots_cles_text,
    nb_docs_slider,
    bouton_recherche,
    sortie_resultats
])

display(interface_bouton)

## 3.5 - Fonction clique_bouton

In [15]:
def clique_bouton(b):
    requete = mots_cles_text.value
    nb_resultats = nb_docs_slider.value
    
    sortie_resultats.clear_output()
    
    with sortie_resultats:
        if not requete.strip():
            print(" Veuillez entrer des mots clés.")
            return
        
        print(f" Recherche : '{requete}'")
        print(f" Nombre de résultats : {nb_resultats}")
        print("\n" + "="*80 + "\n")
        
        try:
            resultats = moteur.search(requete, nb_resultats=nb_resultats)
            
            if len(resultats) > 0:
                print(f" {len(resultats)} résultats trouvés :\n")
                
                for idx, row in resultats.iterrows():
                    print(f"  Résultat {idx+1}")
                    print(f"   Type: [{row['type']}]")
                    print(f"   Titre: {row['titre']}")
                    print(f"   Auteur: {row['auteur']}")
                    print(f"   Score: {row['score']:.4f}")
                    print(f"   URL: {row['url']}")
                    print("\n" + "-"*80 + "\n")
                
                print("\n Tableau récapitulatif :\n")
                display(resultats[['titre', 'auteur', 'score', 'type']])
                
            else:
                print(" Aucun résultat trouvé.")
                
        except Exception as e:
            print(f" Erreur : {str(e)}")

bouton_recherche.on_click(clique_bouton)
print("Fonction configurée !")

Fonction configurée !


## 3.6 - Interface complète

In [16]:
print("Interface de recherche complète :")
print("Entrez vos mots clés et cliquez sur 'Rechercher'\n")

interface_complete = widgets.VBox([
    titre_label,
    widgets.HTML(value="<hr>"),
    mots_cles_text,
    nb_docs_slider,
    bouton_recherche,
    widgets.HTML(value="<hr>"),
    sortie_resultats
], layout=widgets.Layout(padding='10px', border='2px solid #ccc'))

display(interface_complete)

Interface de recherche complète :
Entrez vos mots clés et cliquez sur 'Rechercher'



## 3.7 - Interface avancée avec filtres 

In [17]:
# Interface avancée avec filtres
auteurs_uniques = ['Tous'] + sorted(list(set([doc.auteur for doc in corpus_discours.id2doc.values()])))

titre_avance = widgets.Label(value='Moteur de recherche avancé')

filtre_auteur = widgets.Dropdown(
    options=auteurs_uniques,
    value='Tous',
    description='Auteur :'
)

filtre_type = widgets.Dropdown(
    options=['Tous', 'Reddit', 'Arxiv', 'Document'],
    value='Tous',
    description='Type :'
)

mots_cles_avance = widgets.Text(
    placeholder='mot1 mot2 mot3',
    description='Mots clés :',
    layout=widgets.Layout(width='500px')
)

nb_docs_avance = widgets.IntSlider(
    value=10,
    min=1,
    max=50,
    description="Nombre :",
    layout=widgets.Layout(width='500px')
)

bouton_avance = widgets.Button(
    description='Rechercher',
    button_style='success',
    icon='search'
)

sortie_avance = widgets.Output()

def recherche_avancee(b):
    requete = mots_cles_avance.value
    nb_resultats = nb_docs_avance.value
    auteur_filtre = filtre_auteur.value
    type_filtre = filtre_type.value
    
    sortie_avance.clear_output()
    
    with sortie_avance:
        if not requete.strip():
            print(" Entrez des mots clés.")
            return
        
        print(f"  Recherche avancée")
        print(f"   Requête : '{requete}'")
        print(f"   Filtre auteur : {auteur_filtre}")
        print(f"   Filtre type : {type_filtre}")
        print("\n" + "="*80 + "\n")
        
        try:
            resultats = moteur.search(requete, nb_resultats=nb_resultats*3)
            
            # return if no results
            if len(resultats) == 0:
                print(" Aucun résultat.")
                return
            
            if auteur_filtre != 'Tous':
                resultats = resultats[resultats['auteur'] == auteur_filtre]
            
            if type_filtre != 'Tous':
                resultats = resultats[resultats['type'] == type_filtre]
            
            resultats = resultats.head(nb_resultats)
            
            if len(resultats) > 0:
                print(f" {len(resultats)} résultats après filtrage :\n")
                
                for idx, row in resultats.iterrows():
                    print(f"  {idx+1}. [{row['type']}] {row['titre']}")
                    print(f"   Auteur: {row['auteur']}")
                    print(f"   Score: {row['score']:.4f}\n")
                
                print("\n Tableau :")
                display(resultats[['titre', 'auteur', 'score', 'type']])
            else:
                print(" Aucun résultat.")
                
        except Exception as e:
            print(f" Erreur : {e}")

bouton_avance.on_click(recherche_avancee)

interface_avancee = widgets.VBox([
    titre_avance,
    widgets.HTML(value="<hr>"),
    mots_cles_avance,
    widgets.HBox([filtre_auteur, filtre_type]),
    nb_docs_avance,
    bouton_avance,
    widgets.HTML(value="<hr>"),
    sortie_avance
], layout=widgets.Layout(padding='15px', border='3px solid #4CAF50'))

print("Interface avancée avec filtres :")
display(interface_avancee)

Interface avancée avec filtres :


In [18]:
# Préparation des données temporelles avec type de document
data = []

for doc in corpus_discours.id2doc.values():
    if doc.date is not None:
        data.append({
            "date": pd.to_datetime(doc.date),
            "texte": corpus_discours.nettoyer_texte(doc.texte),
            "type": doc.getType()
        })

df_time = pd.DataFrame(data)
df_time["year_month"] = df_time["date"].dt.to_period("M")

print(f"Documents par type:")
print(df_time["type"].value_counts())
df_time.head()

Documents par type:
Arxiv     183
Reddit    109
Name: type, dtype: int64


,date,texte,type,year_month
0,2025-10-29 01:46:28,i switched to machine learning and i am lost h...,Reddit,2025-10
1,2025-10-29 01:46:28,i m in a year computer science program,Reddit,2025-10
2,2025-10-29 01:46:28,the first years cover general programming and ...,Reddit,2025-10
3,2025-10-29 01:46:28,we had two specializations software and networ...,Reddit,2025-10
4,2025-10-29 01:46:28,i found this really exciting so even after lea...,Reddit,2025-10


In [19]:
def evolution_mot_par_type(df, mot, doc_type):
    """Calcule l'évolution d'un mot par mois pour un type de document"""
    mot = mot.lower()
    df_filtered = df[df["type"] == doc_type]
    
    freq_par_mois = defaultdict(int)
    total_par_mois = defaultdict(int)

    for _, row in df_filtered.iterrows():
        mots = row["texte"].split()
        mois = row["year_month"]

        total_par_mois[mois] += len(mots)
        freq_par_mois[mois] += mots.count(mot)

    resultats = []
    for mois in sorted(freq_par_mois.keys()):
        freq_relative = 0
        if total_par_mois[mois] > 0:
            freq_relative = freq_par_mois[mois] / total_par_mois[mois]

        resultats.append({
            "mois": mois.to_timestamp(),
            "fréquence": freq_relative
        })

    return pd.DataFrame(resultats)

In [20]:
# Widgets pour l'analyse temporelle
mot_temps = widgets.Text(
    description="Mot :",
    placeholder="ex: learning"
)

# obtenir la plage de dates disponibles
all_months = sorted(df_time["year_month"].unique())
month_options = [(m.strftime("%Y.%m"), i) for i, m in enumerate(all_months)]

# Slider pour la période (avec dates réelles)
periode_slider = widgets.SelectionRangeSlider(
    options=month_options,
    index=(0, len(month_options)-1),
    description='Période:',
    continuous_update=False,
    layout=widgets.Layout(width='600px')
)

bouton_temps = widgets.Button(
    description="Évolution temporelle",
    button_style="info"
)

sortie_temps = widgets.Output()

print(f"Période disponible: {all_months[0]} à {all_months[-1]}")

Période disponible: 2017-05 à 2025-12


In [21]:
def afficher_evolution(b):
    with sortie_temps:
        sortie_temps.clear_output()

        mot = mot_temps.value.strip()
        if not mot:
            print("Veuillez entrer un mot.")
            return

        # Récupérer les données pour Arxiv et Reddit
        df_arxiv = evolution_mot_par_type(df_time, mot, "Arxiv")
        df_reddit = evolution_mot_par_type(df_time, mot, "Reddit")

        if df_arxiv.empty and df_reddit.empty:
            print("Aucune donnée disponible.")
            return

        # Filtrer par période sélectionnée
        idx_min, idx_max = periode_slider.value
        mois_min = all_months[idx_min]
        mois_max = all_months[idx_max]
        
        if not df_arxiv.empty:
            df_arxiv = df_arxiv[(df_arxiv["mois"] >= mois_min.to_timestamp()) & 
                                (df_arxiv["mois"] <= mois_max.to_timestamp())]
        if not df_reddit.empty:
            df_reddit = df_reddit[(df_reddit["mois"] >= mois_min.to_timestamp()) & 
                                  (df_reddit["mois"] <= mois_max.to_timestamp())]

        # Créer deux graphiques empilés verticalement
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

        # Graphique Arxiv
        if not df_arxiv.empty:
            ax1.plot(df_arxiv["mois"], df_arxiv["fréquence"], marker="o", color="#FF5722", linewidth=2)
            ax1.fill_between(df_arxiv["mois"], df_arxiv["fréquence"], alpha=0.3, color="#FF5722")
            ax1.set_title(f"Arxiv - '{mot}'", fontsize=12, fontweight='bold')
            ax1.set_ylabel("Fréquence relative")
            ax1.tick_params(axis='x', rotation=45)
            ax1.grid(True, alpha=0.3)
        else:
            ax1.text(0.5, 0.5, "Pas de données Arxiv", ha='center', va='center', fontsize=14)
            ax1.set_title("Arxiv")

        # Graphique Reddit
        if not df_reddit.empty:
            ax2.plot(df_reddit["mois"], df_reddit["fréquence"], marker="o", color="#2196F3", linewidth=2)
            ax2.fill_between(df_reddit["mois"], df_reddit["fréquence"], alpha=0.3, color="#2196F3")
            ax2.set_title(f"Reddit - '{mot}'", fontsize=12, fontweight='bold')
            ax2.set_xlabel("Mois")
            ax2.set_ylabel("Fréquence relative")
            ax2.tick_params(axis='x', rotation=45)
            ax2.grid(True, alpha=0.3)
        else:
            ax2.text(0.5, 0.5, "Pas de données Reddit", ha='center', va='center', fontsize=14)
            ax2.set_title("Reddit")

        plt.tight_layout()
        plt.show()

bouton_temps.on_click(afficher_evolution)

In [22]:
interface_temps = widgets.VBox([
    widgets.HTML("<h3 style='color: #2196F3;'>Analyse temporelle du vocabulaire</h3>"),
    widgets.HTML("<p>Compare l'évolution d'un mot entre Arxiv et Reddit par mois</p>"),
    mot_temps,
    widgets.HTML("<p><b>Sélectionnez la période (glissez pour ajuster):</b></p>"),
    periode_slider,
    bouton_temps,
    sortie_temps
], layout=widgets.Layout(
    padding='15px',
    border='3px solid #2196F3'
))

display(interface_temps)

---
# Analyse avancée du vocabulaire (TF-IDF et BM25)

Cette section implémente les mesures **TF-IDF** et **OKAPI-BM25** pour analyser l'importance des mots dans le corpus.

In [23]:
# Imports supplémentaires pour TF-IDF et BM25
import math
from collections import Counter

In [24]:
class AnalyseurVocabulaire:
    """Classe pour analyser le vocabulaire avec TF-IDF et BM25"""
    
    def __init__(self, corpus):
        self.corpus = corpus
        self.docs = list(corpus.id2doc.values())
        self.N = len(self.docs)  # Nombre total de documents
        self.avgdl = 0  # Longueur moyenne des documents
        self.df = {}  # Document frequency pour chaque mot
        self.idf = {}  # IDF pour chaque mot
        
        self._calculer_stats()
    
    def _calculer_stats(self):
        """Calcule les statistiques nécessaires (DF, IDF, avgdl)"""
        total_length = 0
        doc_freq = Counter()
        
        for doc in self.docs:
            texte_nettoye = self.corpus.nettoyer_texte(doc.texte)
            mots = texte_nettoye.split()
            total_length += len(mots)
            
            # Document frequency : nombre de docs contenant chaque mot
            mots_uniques = set(mots)
            for mot in mots_uniques:
                doc_freq[mot] += 1
        
        self.avgdl = total_length / self.N if self.N > 0 else 0
        self.df = doc_freq
        
        # Calcul de l'IDF
        for mot, freq in self.df.items():
            self.idf[mot] = math.log((self.N - freq + 0.5) / (freq + 0.5) + 1)
    
    def calculer_tf(self, texte):
        """Calcule la fréquence des termes (TF) dans un texte"""
        mots = self.corpus.nettoyer_texte(texte).split()
        tf = Counter(mots)
        total_mots = len(mots)
        
        # Normalisation par la longueur du document
        for mot in tf:
            tf[mot] = tf[mot] / total_mots if total_mots > 0 else 0
        
        return tf
    
    def calculer_tfidf(self, texte):
        """Calcule TF-IDF pour un texte"""
        tf = self.calculer_tf(texte)
        tfidf = {}
        
        for mot, tf_value in tf.items():
            idf_value = self.idf.get(mot, 0)
            tfidf[mot] = tf_value * idf_value
        
        return tfidf
    
    def calculer_bm25(self, texte, k1=1.5, b=0.75):
        """
        Calcule le score BM25 pour un texte
        k1 : paramètre de saturation du terme (typiquement entre 1.2 et 2.0)
        b : paramètre de normalisation de longueur (typiquement 0.75)
        """
        mots = self.corpus.nettoyer_texte(texte).split()
        tf = Counter(mots)
        dl = len(mots)  # longueur du document
        
        bm25_scores = {}
        
        for mot, freq in tf.items():
            if mot in self.idf:
                # Formule BM25
                idf = self.idf[mot]
                numerateur = freq * (k1 + 1)
                denominateur = freq + k1 * (1 - b + b * (dl / self.avgdl))
                bm25_scores[mot] = idf * (numerateur / denominateur)
        
        return bm25_scores
    
    def top_mots_tfidf(self, texte, n=10):
        """Retourne les n mots avec les scores TF-IDF les plus élevés"""
        tfidf = self.calculer_tfidf(texte)
        return sorted(tfidf.items(), key=lambda x: x[1], reverse=True)[:n]
    
    def top_mots_bm25(self, texte, n=10):
        """Retourne les n mots avec les scores BM25 les plus élevés"""
        bm25 = self.calculer_bm25(texte)
        return sorted(bm25.items(), key=lambda x: x[1], reverse=True)[:n]
    
    def comparer_corpus(self, texte1, texte2, n=20):
        """
        Compare deux textes (ou sous-corpus) et identifie :
        - Mots communs
        - Mots spécifiques au texte 1
        - Mots spécifiques au texte 2
        """
        tfidf1 = self.calculer_tfidf(texte1)
        tfidf2 = self.calculer_tfidf(texte2)
        
        mots1 = set(tfidf1.keys())
        mots2 = set(tfidf2.keys())
        
        # Mots communs
        communs = mots1.intersection(mots2)
        communs_scores = [(mot, tfidf1[mot], tfidf2[mot]) for mot in communs]
        communs_scores.sort(key=lambda x: x[1] + x[2], reverse=True)
        
        # Mots spécifiques
        specifiques1 = [(mot, tfidf1[mot]) for mot in mots1 - mots2]
        specifiques1.sort(key=lambda x: x[1], reverse=True)
        
        specifiques2 = [(mot, tfidf2[mot]) for mot in mots2 - mots1]
        specifiques2.sort(key=lambda x: x[1], reverse=True)
        
        return {
            'communs': communs_scores[:n],
            'specifiques_1': specifiques1[:n],
            'specifiques_2': specifiques2[:n]
        }

In [25]:
# Initialisation de l'analyseur de vocabulaire
print("Initialisation de l'analyseur de vocabulaire...")
analyseur = AnalyseurVocabulaire(corpus_discours)
print(f"✓ Analyseur prêt ! (N={analyseur.N} docs, avgdl={analyseur.avgdl:.2f} mots)")

Initialisation de l'analyseur de vocabulaire...
✓ Analyseur prêt ! (N=292 docs, avgdl=20.42 mots)


In [26]:
# ============================================================
# INTERFACE : Analyse TF-IDF et BM25 d'un document
# ============================================================

titre_analyse = widgets.HTML(
    value="<h3 style='color: #FF5722;'> Analyse TF-IDF et BM25</h3>"
)

texte_analyse = widgets.Textarea(
    placeholder="Collez un texte à analyser ici...",
    description="Texte :",
    layout=widgets.Layout(width='100%', height='150px')
)

nb_mots_top = widgets.IntSlider(
    value=10,
    min=5,
    max=30,
    step=5,
    description="Top mots :",
    style={'description_width': '100px'}
)

bouton_analyse = widgets.Button(
    description="Analyser",
    button_style="warning",
    icon='search'
)

sortie_analyse = widgets.Output()

def analyser_document(b):
    with sortie_analyse:
        sortie_analyse.clear_output()
        
        texte = texte_analyse.value.strip()
        if not texte:
            print(" Veuillez entrer un texte à analyser.")
            return
        
        n = nb_mots_top.value
        
        print(" Analyse en cours...\n")
        
        # TF-IDF
        print("=" * 60)
        print(" TOP MOTS PAR TF-IDF")
        print("=" * 60)
        top_tfidf = analyseur.top_mots_tfidf(texte, n)
        for i, (mot, score) in enumerate(top_tfidf, 1):
            print(f"{i:2d}. {mot:20s} → {score:.4f}")
        
        # BM25
        print("\n" + "=" * 60)
        print(" TOP MOTS PAR BM25")
        print("=" * 60)
        top_bm25 = analyseur.top_mots_bm25(texte, n)
        for i, (mot, score) in enumerate(top_bm25, 1):
            print(f"{i:2d}. {mot:20s} → {score:.4f}")
        
        # Statistiques
        mots = corpus_discours.nettoyer_texte(texte).split()
        print("\n" + "=" * 60)
        print(" STATISTIQUES")
        print("=" * 60)
        print(f"Nombre de mots : {len(mots)}")
        print(f"Mots uniques : {len(set(mots))}")
        print(f"Longueur moyenne corpus : {analyseur.avgdl:.2f} mots")

bouton_analyse.on_click(analyser_document)

interface_analyse = widgets.VBox([
    titre_analyse,
    widgets.HTML(value="<hr>"),
    texte_analyse,
    nb_mots_top,
    bouton_analyse,
    widgets.HTML(value="<hr>"),
    sortie_analyse
], layout=widgets.Layout(padding='15px', border='3px solid #FF5722'))

print("Interface d'analyse TF-IDF/BM25 :")
display(interface_analyse)

Interface d'analyse TF-IDF/BM25 :


In [27]:
# ============================================================
# INTERFACE : Comparaison de deux corpus
# ============================================================

titre_comparaison = widgets.HTML(
    value="<h3 style='color: #9C27B0;'>Comparaison de deux corpus</h3>"
)

# Sélection du critère de division
critere_division = widgets.Dropdown(
    options=['auteur', 'type'],
    value='type',
    description="Diviser par :",
    style={'description_width': '100px'}
)

# Widgets pour sélectionner les deux groupes à comparer
groupe1_select = widgets.Dropdown(
    description="Groupe 1 :",
    style={'description_width': '100px'}
)

groupe2_select = widgets.Dropdown(
    description="Groupe 2 :",
    style={'description_width': '100px'}
)

nb_mots_comparaison = widgets.IntSlider(
    value=15,
    min=5,
    max=30,
    step=5,
    description="Top mots :",
    style={'description_width': '100px'}
)

bouton_comparer = widgets.Button(
    description="Comparer",
    button_style="success"
)

sortie_comparaison = widgets.Output()

def mettre_a_jour_groupes(change):
    """Met à jour les options des dropdowns selon le critère"""
    critere = critere_division.value
    
    if critere == 'auteur':
        auteurs = set()
        for doc in corpus_discours.id2doc.values():
            if doc.auteur:
                auteurs.add(doc.auteur)
        options = sorted(list(auteurs))
    elif critere == 'type':
        types = set()
        for doc in corpus_discours.id2doc.values():
            if hasattr(doc, 'type') and doc.type:
                types.add(doc.type)
        options = sorted(list(types))
    else:
        options = []
    
    groupe1_select.options = options
    groupe2_select.options = options
    if len(options) >= 2:
        groupe1_select.value = options[0]
        groupe2_select.value = options[1] if len(options) > 1 else options[0]

# Initialiser les options
mettre_a_jour_groupes(None)
critere_division.observe(mettre_a_jour_groupes, names='value')

def comparer_groupes(b):
    with sortie_comparaison:
        sortie_comparaison.clear_output()
        
        critere = critere_division.value
        groupe1 = groupe1_select.value
        groupe2 = groupe2_select.value
        n = nb_mots_comparaison.value
        
        if not groupe1 or not groupe2:
            print("Veuillez sélectionner deux groupes.")
            return
        
        if groupe1 == groupe2:
            print("Veuillez sélectionner deux groupes différents.")
            return
        
        print(f"Comparaison : {groupe1} vs {groupe2}\n")
        
        # Créer deux textes à partir des documents
        texte1_parts = []
        texte2_parts = []
        
        for doc in corpus_discours.id2doc.values():
            if critere == 'auteur' and doc.auteur == groupe1:
                texte1_parts.append(doc.texte)
            elif critere == 'auteur' and doc.auteur == groupe2:
                texte2_parts.append(doc.texte)
            elif critere == 'type' and hasattr(doc, 'type') and doc.type == groupe1:
                texte1_parts.append(doc.texte)
            elif critere == 'type' and hasattr(doc, 'type') and doc.type == groupe2:
                texte2_parts.append(doc.texte)
        
        texte1 = " ".join(texte1_parts)
        texte2 = " ".join(texte2_parts)
        
        if not texte1 or not texte2:
            print("Un des groupes ne contient aucun document.")
            return
        
        # Comparaison
        resultats = analyseur.comparer_corpus(texte1, texte2, n)
        
        # Affichage des résultats
        print("=" * 70)
        print(f"MOTS SPÉCIFIQUES À : {groupe1}")
        print("=" * 70)
        for i, (mot, score) in enumerate(resultats['specifiques_1'], 1):
            print(f"{i:2d}. {mot:25s} → TF-IDF: {score:.4f}")
        
        print("\n" + "=" * 70)
        print(f"MOTS SPÉCIFIQUES À : {groupe2}")
        print("=" * 70)
        for i, (mot, score) in enumerate(resultats['specifiques_2'], 1):
            print(f"{i:2d}. {mot:25s} → TF-IDF: {score:.4f}")
        
        print("\n" + "=" * 70)
        print("MOTS COMMUNS (scores combinés)")
        print("=" * 70)
        for i, (mot, score1, score2) in enumerate(resultats['communs'], 1):
            total = score1 + score2
            print(f"{i:2d}. {mot:25s} → {groupe1}: {score1:.4f} | {groupe2}: {score2:.4f}")
        
        # Statistiques
        mots1 = corpus_discours.nettoyer_texte(texte1).split()
        mots2 = corpus_discours.nettoyer_texte(texte2).split()
        
        print("\n" + "=" * 70)
        print("STATISTIQUES")
        print("=" * 70)
        print(f"{groupe1:25s} → {len(texte1_parts)} docs, {len(mots1)} mots, {len(set(mots1))} mots uniques")
        print(f"{groupe2:25s} → {len(texte2_parts)} docs, {len(mots2)} mots, {len(set(mots2))} mots uniques")

bouton_comparer.on_click(comparer_groupes)

interface_comparaison = widgets.VBox([
    titre_comparaison,
    widgets.HTML(value="<hr>"),
    critere_division,
    widgets.HBox([groupe1_select, groupe2_select]),
    nb_mots_comparaison,
    bouton_comparer,
    widgets.HTML(value="<hr>"),
    sortie_comparaison
], layout=widgets.Layout(padding='15px', border='3px solid #9C27B0'))

print("Interface de comparaison de corpus :")
display(interface_comparaison)


Interface de comparaison de corpus :


In [28]:
# ============================================================
# VISUALISATION : Graphiques TF-IDF et BM25
# ============================================================

def visualiser_scores(texte, n=15):
    """Crée des graphiques pour comparer TF-IDF et BM25"""
    
    top_tfidf = analyseur.top_mots_tfidf(texte, n)
    top_bm25 = analyseur.top_mots_bm25(texte, n)
    
    # Extraction des données
    mots_tfidf = [mot for mot, _ in top_tfidf]
    scores_tfidf = [score for _, score in top_tfidf]
    
    mots_bm25 = [mot for mot, _ in top_bm25]
    scores_bm25 = [score for _, score in top_bm25]
    
    # Création des graphiques
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # TF-IDF
    ax1.barh(range(len(mots_tfidf)), scores_tfidf, color='#2196F3')
    ax1.set_yticks(range(len(mots_tfidf)))
    ax1.set_yticklabels(mots_tfidf)
    ax1.set_xlabel('Score TF-IDF', fontsize=12)
    ax1.set_title('Top mots par TF-IDF', fontsize=14, fontweight='bold')
    ax1.invert_yaxis()
    ax1.grid(axis='x', alpha=0.3)
    
    # BM25
    ax2.barh(range(len(mots_bm25)), scores_bm25, color='#FF5722')
    ax2.set_yticks(range(len(mots_bm25)))
    ax2.set_yticklabels(mots_bm25)
    ax2.set_xlabel('Score BM25', fontsize=12)
    ax2.set_title('Top mots par BM25', fontsize=14, fontweight='bold')
    ax2.invert_yaxis()
    ax2.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Widget pour tester la visualisation
bouton_visu = widgets.Button(
    description=" Visualiser les scores",
    button_style="info"
)

sortie_visu = widgets.Output()

def afficher_visualisation(b):
    with sortie_visu:
        sortie_visu.clear_output()
        
        texte = texte_analyse.value.strip()
        if not texte:
            print(" Veuillez d'abord entrer un texte dans l'interface d'analyse ci-dessus.")
            return
        
        visualiser_scores(texte, nb_mots_top.value)

bouton_visu.on_click(afficher_visualisation)

display(widgets.VBox([
    bouton_visu,
    sortie_visu
]))